[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CU-Robotics/swarm/blob/main/cnn/hyper_parameter_optimization.ipynb)

In [ ]:
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from torch.utils.data import Dataset

from PIL import Image
from tqdm import tqdm
import os
import json
from datetime import datetime

In [6]:
if 'COLAB_GPU' in os.environ or 'TENSORFLOW_USE_SYNC_ON_FINISH' in os.environ:
    print("Running in Google Colab")
    # Download the zip file from Google Drive
    !gdown --fuzzy https://drive.google.com/file/d/1yyGVx6jvZgLLSX4yx9c8JBKDM5bTa9eK/view?usp=drive_link
    !unzip -q data.zip -d /content/data/
    !pip install optuna
    on_colab = True

else:
    print("Not running in Google Colab (likely VS Code or local environment)")
    on_colab = False

Not running in Google Colab (likely VS Code or local environment)


In [7]:
import optuna
from optuna.trial import TrialState

DEVICE = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
BATCHSIZE = 128
CLASSES = 7

if on_colab:
    DIR = os.getcwd() + "/data/"
else:
    DIR = os.getcwd() + "/../collections/data/"
    # import your own data :p

EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

print(f'Using device: {DEVICE}')

Using device: cpu


c:\Users\geodz\anaconda3\envs\MachineLearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Custom dataset class for loading images and labels
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = json.load(open(annotations_file)) # path to pipeline.json
        self.img_dir = img_dir                              # folder where the cleaned images are
        self.transform = transform
        self.classes = {'1':1, '2':2, '3':3, '4':4, 'sentry':5, 'base':6, 'tower':7}

    def __len__(self):
        return len(self.img_labels)

    # gets image and label at index idx based on position in json
    def __getitem__(self, idx):
        image_data = self.img_labels[idx]
        img_name = image_data["name"]
        img_folder = image_data["folder"]

        img_path = os.path.join(self.img_dir, img_folder, "cropped", img_name)

        image = Image.open(img_path).convert("RGB")

        label = self.classes[image_data["labels"]["icon"]]

        if self.transform:
            image = self.transform(image)

        return image, label



In [9]:
def define_model(trial):

    num_layers = trial.suggest_int('num_layers', 1, 3)
    layers = []

    in_features = 1
    for i in range(num_layers):
        out_features = trial.suggest_int(f'n_units_l{i}', 16, 64, step = 16)
        kernel_size = trial.suggest_int(f'kernel_size_l{i}', 3, 7, step=2)
        layers.append(nn.Conv2d(in_features, out_features, kernel_size=kernel_size, padding=kernel_size//2))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(2))

        in_features = out_features

    model = nn.Sequential(*layers)

    # Flatten layer
    model.add_module("flatten", nn.Flatten())

    # Estimate feature size after convolutions
    with torch.no_grad():
        dummy = torch.zeros(1, 1, 100, 100)
        n_features = model(dummy).shape[1]

    # Add final classifier
    model.add_module("fc", nn.Linear(n_features, CLASSES))
    model.add_module("logsoftmax", nn.LogSoftmax(dim=1))

    return model

In [10]:
def get_data_loaders():
    annotations_file = os.path.join(DIR, 'cleaned_metadata.json')

    # transform = transforms.Compose([
    #     transforms.Grayscale(num_output_channels=1),
    #     transforms.ToTensor(),
    #     transforms.Lambda(lambda t: t.sqrt()),
    # ])

    transform = transforms.Compose([
      transforms.Grayscale(num_output_channels=1),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(15),
      transforms.ColorJitter(brightness=0.2, contrast=0.2),
      transforms.ToTensor(),
      transforms.Lambda(lambda t: t.sqrt()),
    ])

    dataset = CustomImageDataset(annotations_file, img_dir=DIR, transform=transform)

    # Split sizes
    train_size = int(0.8 * len(dataset))  # 80%
    val_size = len(dataset) - train_size  # remaining 20%

    # Random split
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=BATCHSIZE, shuffle=True, num_workers=2)

    val_loader = DataLoader(val_dataset, batch_size=BATCHSIZE, shuffle=True, num_workers=2)

    return train_loader, val_loader

In [11]:

def objective(trial):
    model = define_model(trial).to(DEVICE)

    # Suggest hyperparameters for optimizer
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, valid_loader = get_data_loaders()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), 1, 100, 100).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), 1, 100, 100).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [12]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=1000)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-10-28 16:12:26,860] A new study created in memory with name: no-name-bc9f26c4-6a7a-47e1-8379-359aa7163ab5
[W 2025-10-28 16:12:26,873] Trial 0 failed with parameters: {'num_layers': 1, 'n_units_l0': 48, 'kernel_size_l0': 3, 'lr': 3.7766934659193476e-05, 'optimizer': 'Adam'} because of the following error: FileNotFoundError(2, 'No such file or directory').
Traceback (most recent call last):
  File "c:\Users\geodz\anaconda3\envs\MachineLearning\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\geodz\AppData\Local\Temp\ipykernel_28908\2896430027.py", line 9, in objective
    train_loader, valid_loader = get_data_loaders()
                                 ^^^^^^^^^^^^^^^^^^
  File "C:\Users\geodz\AppData\Local\Temp\ipykernel_28908\3575368090.py", line 19, in get_data_loaders
    dataset = CustomImageDataset(annotations_file, img_dir=DIR, transform=transform)
              ^^^^^^^

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\geodz\\OneDrive\\Documents\\School\\Robotics Club\\swarm\\cnn/../collections/data/cleaned_metadata.json'

In [ ]:
import pandas as pd

# Convert trials to DataFrame
df = study.trials_dataframe()
# print(df)

# Save to CSV
df.to_csv("optuna_trials_rand.csv", index=False)